In [442]:
import pandas as pd
import numpy as np
import seaborn as sb
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from sklearn.compose import TransformedTargetRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelBinarizer
import seaborn as sns
from sklearn.feature_extraction.text import CountVectorizer
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import string
import contractions
from num2words import num2words
from textblob import TextBlob
# from spellchecker import SpellChecker
from gensim.models import Word2Vec
import re
# from bs4 import BeautifulSoup
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
# train['SEO Description'] = train['SEO Description'].astype(str)
# train['SEO Description'] = train['SEO Description'].str.lower()

pd.options.display.max_columns = None
pd.options.display.max_rows = None

# Load the dataset
df = pd.read_excel('/users/amulya/Desktop/Tam_2904.xlsx')
df = df.fillna(0)
df.head(1)

[nltk_data] Downloading package punkt to /Users/amulya/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/amulya/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/amulya/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


,Company,Lists,Employees,Industry,Company City,Company State,Company Country,Keywords,SEO Description,Technologies,Total Funding,Latest Funding,Latest Funding Amount,Last Raised At,Annual Revenue,Number of Retail Locations,Short Description,Founded Year,ICP
0,Gainsight,"Brain 30K SAAS USA, data companies 24, Tarun D...",1300.0,information technology & services,San Francisco,California,United States,"enterprise software, saas, predictive analytic...",Gainsight's Customer Success and Product Exper...,"Salesforce, UltraDns, Sendgrid, Gmail, Mimecas...",156000000,Series E,52000000,2017-05-09 00:00:00,98000000.0,0.0,"At Gainsight, our mission is to be living proo...",2009,1


In [443]:
train, test = train_test_split(df,test_size=0.3, random_state=42)
print("Training set size:", len(train))
print("Testing set size:", len(test))
print("set size:", len(df))

Training set size: 2748
Testing set size: 1178
set size: 3926


In [444]:
company_backup_test = test['Company'].copy()
company_backup_test.info()

<class 'pandas.core.series.Series'>
Index: 1178 entries, 3476 to 3793
Series name: Company
Non-Null Count  Dtype 
--------------  ----- 
1178 non-null   object
dtypes: object(1)
memory usage: 18.4+ KB


In [445]:
def pre_process_1(df):
    icp_train_backup = df['ICP'].copy()
    company_train_backup = df['Company'].copy()
    df_dummies = pd.DataFrame({'Employees': df['Employees']})
    # Create binary columns based on employee categories
    df_dummies['emp_lessthan50'] = np.where(df_dummies['Employees'] <= 50, 1, 0)
    df_dummies['emp_50to100'] = np.where((df_dummies['Employees'] > 50) & (df_dummies['Employees'] <= 100), 1, 0)
    df_dummies['emp_100_above'] = np.where(df_dummies['Employees'] > 100, 1, 0)
    df = pd.concat([df, df_dummies], axis=1)  
    df['SEO Description'] = df['SEO Description'].replace(0, 'Others')
    df['SEO Description'] = df['SEO Description'].astype(str)
    df['SEO Description'] = df['SEO Description'].str.lower()
    
    df['SEO Description'] = df['SEO Description'].apply(word_tokenize)
    punctuation = set(string.punctuation)
    df['SEO Description'] = df['SEO Description'].apply(lambda tokens: [token for token in tokens if token not in punctuation])

    stop_words = set(stopwords.words('english'))
    df['SEO Description'] = df['SEO Description'].apply(lambda tokens: [token for token in tokens if token not in stop_words])

    lemmatizer = WordNetLemmatizer()
    df['SEO Description'] = df['SEO Description'].apply(lambda tokens: [lemmatizer.lemmatize(token) for token in tokens])

    df['SEO Description'] = df['SEO Description'].apply(lambda tokens: [num2words(token) if token.isdigit() else token for token in tokens])

    df['SEO Description'] = df['SEO Description'].apply(lambda tokens: [contractions.fix(token) for token in tokens])

    df['SEO Description'] = df['SEO Description'].apply(lambda tokens: [token for token in tokens if token.isalnum()])

    df['SEO Description'] = df['SEO Description'].apply(lambda tokens: [token.strip() for token in tokens])

    df['SEO Description'] = df['SEO Description'].apply(lambda tokens: [re.sub(r'http\S+', '', token) for token in tokens])

    df['SEO Description'] = df['SEO Description'].apply(' '.join)
    return df
# train = pre_process_1(train)

In [446]:
def seo_tfidf(df):
    from sklearn.feature_extraction.text import TfidfVectorizer
    import pickle
    from sklearn.preprocessing import LabelBinarizer
    from sklearn.preprocessing import MultiLabelBinarizer
    
    tfidf_vectorizer = TfidfVectorizer()
    tfidf_matrix = tfidf_vectorizer.fit(df['SEO Description'])
    with open('tfidf_vectorizer.pkl', 'wb') as f:
        pickle.dump(tfidf_vectorizer, f)
    ########################
    lb_cities = LabelBinarizer()
    df['Company City'] = df['Company City'].astype(str)
    lb_cities.fit(df['Company City'])
    with open('lb_cities.pkl', 'wb') as f:
        pickle.dump(lb_cities, f) 
    ########################
    df['Technologies'] = df['Technologies'].replace(0, 'Other_technology')
    technologies_lists = df['Technologies'].str.split(',')
    mlb = MultiLabelBinarizer()
    technologies_encoded = mlb.fit(technologies_lists)  
    with open('technologies_encoded.pkl', 'wb') as f:
        pickle.dump(mlb, f)
    ########################
    df['Keywords'] = df['Keywords'].astype(str)
    df['Keywords'] = df['Keywords'].replace(0, 'Other_keyword')
    keywords_lists = df['Keywords'].str.split(',')
    mlb = MultiLabelBinarizer()
    keywords_encoded = mlb.fit(keywords_lists)  
    with open('keywords_encoded.pkl', 'wb') as f:
        pickle.dump(mlb, f) 
    return df

# df = seo_tfidf(df)
# df = seo_process_pickel(df)


In [447]:
def seo_process_pickel(df):
    import pickle
    with open('tfidf_vectorizer.pkl', 'rb') as f:
        loaded_tfidf_vectorizer = pickle.load(f)
    tfidf_matrix = loaded_tfidf_vectorizer.transform(df['SEO Description'])
    feature_names_test = loaded_tfidf_vectorizer.get_feature_names_out()
    seo_done_df = pd.DataFrame(tfidf_matrix.toarray(), columns=feature_names_test)
    df.reset_index(drop=True, inplace=True)
    seo_done_df.reset_index(drop=True, inplace=True)
    df = pd.concat([df, seo_done_df], axis=1)
    ########################
    with open('lb_cities.pkl', 'rb') as f:
        lb_cities = pickle.load(f)
    df['Company City'] = df['Company City'].astype(str)
    cities_proc = lb_cities.transform(df['Company City'])
    cities_df = pd.DataFrame(cities_proc, columns=lb_cities.classes_)
    df.reset_index(drop=True, inplace=True)
    cities_df.reset_index(drop=True, inplace=True)
    df = pd.concat([df, cities_df], axis=1)
    #########################
    with open('technologies_encoded.pkl', 'rb') as f:
        mlb = pickle.load(f)
    df['Technologies'] = df['Technologies'].astype(str)
    technologies_lists = df['Technologies'].str.split(',')
    technologies_encoded = mlb.transform(technologies_lists)
    technologies_encoded_df = pd.DataFrame(technologies_encoded, columns=mlb.classes_)
    df.reset_index(drop=True, inplace=True)
    technologies_encoded_df.reset_index(drop=True, inplace=True)
    df.reset_index(drop=True, inplace=True)
    technologies_encoded_df.reset_index(drop=True, inplace=True)    
    df = pd.concat([df, technologies_encoded_df], axis=1)  
    ###########################
    with open('keywords_encoded.pkl', 'rb') as f:
        mlb = pickle.load(f)
    df['Keywords'] = df['Keywords'].astype(str)
    keywords_lists = df['Keywords'].str.split(',')
    keywords_encoded = mlb.transform(keywords_lists)
    keywords_encoded_df = pd.DataFrame(keywords_encoded, columns=mlb.classes_)
    df.reset_index(drop=True, inplace=True)
    keywords_encoded_df.reset_index(drop=True, inplace=True)

    df.reset_index(drop=True, inplace=True)
    keywords_encoded_df.reset_index(drop=True, inplace=True)
    
    df = pd.concat([df, keywords_encoded_df], axis=1)  
    return df

In [448]:
def drop_columns(df):
    
    df = df.drop(['Company', 'Lists', 'Employees','Company State', 'Company Country', 'SEO Description',
                  'Latest Funding','Industry','Company City','Keywords','Technologies','Last Raised At','Short Description',
                  'Latest Funding Amount','Number of Retail Locations', 'Founded Year','Total Funding','Annual Revenue'], axis = 1)
    # df['ICP'] = df['ICP'].replace({'ICP - 2': 1, 'ICP-1': 1, 'ICP - 1': 1, 'Not an ICP' : 0})
    df['ICP'] = df['ICP'].replace({'S2' : 1, 'S1' : 1, 'S3':1, 'B2' : 1, 'B1' : 1, 'Not Relevant/ Not ICP)': 0,'ICP - 2': 1, 'ICP-1': 1, 'ICP - 1': 1, 'Not an ICP' : 0})

    return df
# df = drop_columns(df)

In [449]:
def all_methods(df):
    train_new = pre_process_1(df)
    train_new = seo_tfidf(train_new)
    train_new = seo_process_pickel(train_new)
    train_new = drop_columns(train_new)
    return train_new
train_final = all_methods(train)

In [450]:
train_final.head(1)

,ICP,emp_lessthan50,emp_50to100,emp_100_above,10x,14daydays,160k,1k,1st,20k,2a,2b,2d,2x,30x,3d,3one4,3pl,4pl,4tellx,5data,5g,5k,5x,8x,aaa,aacsb,aadhaar,ab,abandoned,abandonment,abd,ability,able,abroad,abstract,abundance,academic,academy,accacia,accelarate,accelerate,accelerated,accelerates,accelerating,acceleration,accelerist,accelerize,accentiqa,accept,accepted,accepting,access,accessible,accessory,accomplish,accoona,accops,accord,accordingly,account,accountability,accountable,accounting,accreditation,accruent,accuracy,accurate,acd,ace,acesse,ach,achievable,achieve,achieving,achived,acid,acquire,acquisition,across,act,acting,action,actionable,activation,active,actively,activeprospect,activision,activity,actlogica,actor,actual,actum,acunetix,ad,adapt,adaptable,adaptive,add,added,adding,additionally,address,adherence,aditya,adjust,admin,administered,administration,administrative,admission,adopt,adoption,adsr,adtech,advance,advanced,advancement,advancing,advantage,advenio,adverse,advertisement,advertiser,advertising,advice,advise,advisement,advisor,advisory,advocacy,aeromegh,aerospace,aesthetic,affair,affinity,affordable,age,agen,agency,agent,agentless,aggregate,aggregation,aggregator,aggressively,agile,agility,agnext,agnostic,agreefy,agreement,agri,agribusiness,agriculture,agritech,agvs,ahavaplan,ahead,ahmedabad,ai,aia,aid,aim,aimed,aiming,aiosell,aiproducts,air,airbrick,airport,airtable,aither,akun,alan,alarm,alert,alertfind,alerting,alexa,algorithm,algoryte,algos,algoshack,ali,alif,align,aligned,aligns,alippo,allclear,allegro,allgovision,alliance,allocation,allow,allowing,allows,allyz,alone,along,alphamap,also,alternate,alternative,althi,altura,altus,alumnus,always,alytsee,amazing,amazon,ambit,ambulatory,america,among,amongst,amorphous,amount,amp,amplifies,amplify,amplifying,analysis,analyst,analytic,analytical,analytics,analyze,analyzes,analyzing,anda,android,angeles,angular,angularjs,animal,ankalabs,ankalytik,ankur,announce,anomaly,anonymous,answer,answercart,answering,ant,anterior,anthony,antonio,antshares,anxiety,anyone,anything,anywhere,aov,ap,apart,api,apis,apk,aplikasi,apna,apollo,app,apparel,appbiz360,apple,appliance,applicable,applicant,application,applied,applies,apply,applying,appointment,appointmentcore,appraisal,apprentice,approach,approval,approved,apps,appsian,appspace,appsteer,apty,ar,arabic,arch,architect,architectural,architecture,area,argentina,ariba,arkansas,arm,around,array,art,arth,article,artificial,artisan,artisanal,artist,arzneimittelversorgung,asanify,ascent,ascentfactory,asia,ask,aspartame,aspect,aspiration,aspiring,assert,assessment,asset,assist,assistance,assistant,association,assurance,assure,assured,assures,astonishing,astrea,astro,astrological,astrology,astrologyapi,astronomic,at,athena,athera,athleisure,athlete,atlan,atlas,atlassian,atlasview,atm,attack,attendance,attention,attentionselect,atticous,attify,attorney,attri,attribute,audience,audio,audiovisual,audit,auditbase,auditing,auditmation,augment,augmentation,augmented,august,auraml,aurea,auscultation,austin,australia,australian,authentic,authentication,author,authority,authorized,autify,auto,autocorrect,automate,automated,automates,automatic,automatically,automating,automatiom,automation,automator,automobile,automotive,autonomous,autonomy,autopilot,auxilo,av,avail,available,availing,avataar,avatar,average,avi,aviation,avid,avyst,av无码精品一区二区三区,awakening,award,awarded,aware,awareness,away,awesome,aws,axa,axcelerateai,axis,axonator,axpert,azure,b1,b2b,baby,babyoye,back,backbone,backed,backend,background,backup,badge,bag,bakery,baking,balance,balanced,balancing,ballerina,bamzooka,band,bandwidth,bane,bangalore,bank,banking,banksathi,bar,bare,barrier,base,based,basic,basis,battery,bay,bazel,bba,beautiful,beautifully,beauty,because,become,beginner,behalf,behavior,behavioral,behaviour,belief,believe,bellary,bellevue,belong,benchkart,benchmarking,benefit,bengaluru,berkshire,bespoke,best,bet,beta,better,betterhalf,beverage,beyond,bfsi,bgv,bhopal,bhu

In [451]:
df_no_duplicates = train_final.drop_duplicates()

# Print the shape of the DataFrame before and after removing duplicates
print("Shape of DataFrame before removing duplicates:", train_final.shape)
print("Shape of DataFrame after removing duplicates:", df_no_duplicates.shape)

Shape of DataFrame before removing duplicates: (2748, 16465)
Shape of DataFrame after removing duplicates: (2604, 16465)


In [452]:
df_transposed = train_final.T

# Use drop_duplicates to keep only one instance of each column name
df_transposed = df_transposed.drop_duplicates()

# Transpose the DataFrame back to its original shape
df_unique_columns = df_transposed.T
print("Shape of DataFrame before removing duplicates:", train_final.shape)
print("Shape of DataFrame after removing duplicates:", df_unique_columns.shape)

Shape of DataFrame before removing duplicates: (2748, 16465)
Shape of DataFrame after removing duplicates: (2748, 7551)


In [453]:
duplicate_columns = df_unique_columns.columns.duplicated()

# Print the result
if duplicate_columns.any():
    print("Duplicate columns are present in the DataFrame.")
    # print("Duplicate column names:", df.columns[duplicate_columns].tolist())
else:
    print("No duplicate columns are present in the DataFrame.")

Duplicate columns are present in the DataFrame.


In [454]:
df_unique_columns.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2748 entries, 0 to 2747
Columns: 7551 entries, ICP to zero trust sase
dtypes: float64(7551)
memory usage: 158.3 MB


In [418]:
train_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2748 entries, 0 to 2747
Columns: 16465 entries, ICP to zoho campaigns
dtypes: float64(4950), int64(11515)
memory usage: 345.2 MB


In [408]:
# test = pd.read_excel('/users/amulya/Downloads/9000Austin Dalls - AI_ML Data (1).xlsx')
# # test.rename(columns={'# Employees': 'Employees'}, inplace=True)
# print(test.info())
# test.head(1)
# test['# Employees']
# test['Total Funding'] = test['Total Funding'].fillna(0)

In [455]:
def test_all_methods(df):
    test_new = pre_process_1(df)
    test_new = seo_process_pickel(test_new)
    test_new = drop_columns(test_new)
    return test_new
test_final = test_all_methods(test)

/opt/homebrew/anaconda3/envs/amulya_base/lib/python3.9/site-packages/sklearn/preprocessing/_label.py:900: UserWarning: unknown class(es) [' 123ContactForm', ' AddShoppers', ' Adobe Connect', ' Advertising.com', ' AppDynamics', ' BounceExchange', ' Captify', ' Carbon', ' Channel Advisor', ' Cision', ' Citrix NetScaler', ' ClickDesk', ' DealerOn', ' Dstillery', ' DynamicYield', ' E-planning', ' Ecwid', ' Elucidat', ' Embroker', ' Genoo', ' GeoTrust', ' Gravity', ' Hatchbuck', ' IFS ERP', ' IgnitionOne', ' InContact', ' Insperity', ' Intense Debate', ' Intershop', ' Janrain', ' JobApp', ' JobDiva', ' Klarna', ' LeadFormix', ' LiquidPlanner', ' LivePerson Monitor', ' Loggly', ' MemberClicks', ' MemberPress', ' Mentad', ' Mobileiron', ' Mosyle', ' Muvi', ' Nielsen Display Ads (Formerly eXelate)', ' Oracle Products', ' Podia', ' PrototypeJS', ' RackSpace DNS', ' Rackspace', ' Right Media (Yahoo Ads)', ' RocketFuel', ' Rubicon Project', ' Samcart', ' Saucelabs', ' Sequence Monitor SPF', ' Sil

In [461]:
test_transposed = test_final.T

# Use drop_duplicates to keep only one instance of each column name
test_transposed = test_transposed.drop_duplicates()

# Transpose the DataFrame back to its original shape
test_unique_columns = test_transposed.T
print("Shape of DataFrame before removing duplicates:", test_final.shape)
print("Shape of DataFrame after removing duplicates:", test_unique_columns.shape)

Shape of DataFrame before removing duplicates: (1178, 16465)
Shape of DataFrame after removing duplicates: (1178, 3482)


In [456]:
print(test_final.info())
test_final.head(1)

# test_final = test_final.drop(['Unnamed: 18','Unnamed: 19','Unnamed: 20','Unnamed: 21','Unnamed: 22','Unnamed: 23','Unnamed: 24','Unnamed: 25','Unnamed: 26','Unnamed: 27','Unnamed: 28','Unnamed: 29','Unnamed: 30'], axis = 1)
# test_final.head(1)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1178 entries, 0 to 1177
Columns: 16465 entries, ICP to zoho campaigns
dtypes: float64(4950), int64(11515)
memory usage: 148.0 MB
None


,ICP,emp_lessthan50,emp_50to100,emp_100_above,10x,14daydays,160k,1k,1st,20k,2a,2b,2d,2x,30x,3d,3one4,3pl,4pl,4tellx,5data,5g,5k,5x,8x,aaa,aacsb,aadhaar,ab,abandoned,abandonment,abd,ability,able,abroad,abstract,abundance,academic,academy,accacia,accelarate,accelerate,accelerated,accelerates,accelerating,acceleration,accelerist,accelerize,accentiqa,accept,accepted,accepting,access,accessible,accessory,accomplish,accoona,accops,accord,accordingly,account,accountability,accountable,accounting,accreditation,accruent,accuracy,accurate,acd,ace,acesse,ach,achievable,achieve,achieving,achived,acid,acquire,acquisition,across,act,acting,action,actionable,activation,active,actively,activeprospect,activision,activity,actlogica,actor,actual,actum,acunetix,ad,adapt,adaptable,adaptive,add,added,adding,additionally,address,adherence,aditya,adjust,admin,administered,administration,administrative,admission,adopt,adoption,adsr,adtech,advance,advanced,advancement,advancing,advantage,advenio,adverse,advertisement,advertiser,advertising,advice,advise,advisement,advisor,advisory,advocacy,aeromegh,aerospace,aesthetic,affair,affinity,affordable,age,agen,agency,agent,agentless,aggregate,aggregation,aggregator,aggressively,agile,agility,agnext,agnostic,agreefy,agreement,agri,agribusiness,agriculture,agritech,agvs,ahavaplan,ahead,ahmedabad,ai,aia,aid,aim,aimed,aiming,aiosell,aiproducts,air,airbrick,airport,airtable,aither,akun,alan,alarm,alert,alertfind,alerting,alexa,algorithm,algoryte,algos,algoshack,ali,alif,align,aligned,aligns,alippo,allclear,allegro,allgovision,alliance,allocation,allow,allowing,allows,allyz,alone,along,alphamap,also,alternate,alternative,althi,altura,altus,alumnus,always,alytsee,amazing,amazon,ambit,ambulatory,america,among,amongst,amorphous,amount,amp,amplifies,amplify,amplifying,analysis,analyst,analytic,analytical,analytics,analyze,analyzes,analyzing,anda,android,angeles,angular,angularjs,animal,ankalabs,ankalytik,ankur,announce,anomaly,anonymous,answer,answercart,answering,ant,anterior,anthony,antonio,antshares,anxiety,anyone,anything,anywhere,aov,ap,apart,api,apis,apk,aplikasi,apna,apollo,app,apparel,appbiz360,apple,appliance,applicable,applicant,application,applied,applies,apply,applying,appointment,appointmentcore,appraisal,apprentice,approach,approval,approved,apps,appsian,appspace,appsteer,apty,ar,arabic,arch,architect,architectural,architecture,area,argentina,ariba,arkansas,arm,around,array,art,arth,article,artificial,artisan,artisanal,artist,arzneimittelversorgung,asanify,ascent,ascentfactory,asia,ask,aspartame,aspect,aspiration,aspiring,assert,assessment,asset,assist,assistance,assistant,association,assurance,assure,assured,assures,astonishing,astrea,astro,astrological,astrology,astrologyapi,astronomic,at,athena,athera,athleisure,athlete,atlan,atlas,atlassian,atlasview,atm,attack,attendance,attention,attentionselect,atticous,attify,attorney,attri,attribute,audience,audio,audiovisual,audit,auditbase,auditing,auditmation,augment,augmentation,augmented,august,auraml,aurea,auscultation,austin,australia,australian,authentic,authentication,author,authority,authorized,autify,auto,autocorrect,automate,automated,automates,automatic,automatically,automating,automatiom,automation,automator,automobile,automotive,autonomous,autonomy,autopilot,auxilo,av,avail,available,availing,avataar,avatar,average,avi,aviation,avid,avyst,av无码精品一区二区三区,awakening,award,awarded,aware,awareness,away,awesome,aws,axa,axcelerateai,axis,axonator,axpert,azure,b1,b2b,baby,babyoye,back,backbone,backed,backend,background,backup,badge,bag,bakery,baking,balance,balanced,balancing,ballerina,bamzooka,band,bandwidth,bane,bangalore,bank,banking,banksathi,bar,bare,barrier,base,based,basic,basis,battery,bay,bazel,bba,beautiful,beautifully,beauty,because,become,beginner,behalf,behavior,behavioral,behaviour,belief,believe,bellary,bellevue,belong,benchkart,benchmarking,benefit,bengaluru,berkshire,bespoke,best,bet,beta,better,betterhalf,beverage,beyond,bfsi,bgv,bhopal,bhu

In [382]:
train_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3533 entries, 0 to 3532
Columns: 19682 entries, ICP to zoho campaigns
dtypes: float64(5766), int64(13916)
memory usage: 530.5 MB


In [398]:
train_columns = set(train_final.columns)
test_columns = set(test_final.columns)

if np.array_equal(train_columns, test_columns):
    print("The feature names are the same for train and test data.")
else:
    print("The feature names are different for train and test data.")

missing_columns_in_test = train_columns - test_columns
print(missing_columns_in_test)

The feature names are different for train and test data.
{'ICP'}


In [457]:
def analysis(y,y_pred):
    train_accuracy = accuracy_score(y, y_pred)
    train_precision = precision_score(y, y_pred)
    train_recall = recall_score(y, y_pred)
    train_f1 = f1_score(y, y_pred)
    print(f'accuracy : {train_accuracy}\nprecision:{train_precision}\nrecall:{train_recall}\nf1_score:{train_f1}')
    return 
    

In [458]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix, roc_curve, roc_auc_score

y = df_unique_columns['ICP']
x = df_unique_columns.drop('ICP',axis = 1)
model = LogisticRegression()
model.fit(x, y)
y_pred = model.predict(x)
analysis(y, y_pred)

accuracy : 0.9446870451237264
precision:0.9700996677740864
recall:0.8751248751248751
f1_score:0.9201680672268907


In [459]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix, roc_curve, roc_auc_score

test_backup_final = test_final['ICP'].copy()

y = test_final['ICP']
x = test_final.drop('ICP',axis = 1)
y_pred_test = model.predict(x)
analysis(y, y_pred_test)

ValueError: The feature names should match those that were passed during fit.
Feature names unseen at fit time:
-  1click payments
-  3d animations
-  3d characters modeling animation
-  3d ecommerce
-  3d geometry applications
- ...


In [402]:
# from sklearn.preprocessing import MinMaxScaler
# from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix, roc_curve, roc_auc_score

# # test_backup_final = test_final['ICP'].copy()
# test_final.columns = test_final.columns.astype(str)
# # y = test_final['ICP']
# x = test_final
# y_pred_test = model.predict(x)
# print(y_pred_test)
# test['predicted_output'] = y_pred_test
# test.head()

[0 0 1 ... 0 0 0]


,Company,Lists,Employees,Industry,Company City,Company State,Company Country,Keywords,SEO Description,Technologies,Total Funding,Latest Funding,Latest Funding Amount,Last Raised At,Annual Revenue,Number of Retail Locations,Short Description,Founded Year,predicted_output
0,Southwestern Benefit Designers,NaN,6.0,information technology & services,Austin,Texas,United States,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,PATH Software,"Brain 30K SAAS USA, data companies 24",5.0,information technology & services,Austin,Texas,United States,"solutions for window, door dealerships, busine...",NaN,"Gmail, Google Apps, MailChimp SPF, Google Clou...",NaN,NaN,NaN,NaN,402000.0,NaN,PATH | WD was born out of an observation that ...,2010,0
2,DealHub.io,"Brain 30K SAAS USA, Dhanush/Apollo/Feb 22/2023...",260.0,information technology & services,Austin,Texas,United States,"cpq, proposals, guided selling, sales quoting,...",Streamline your CLM contract management. Win d...,"Cloudflare DNS, Mailchimp Mandrill, Gmail, Goo...",84500000,Series C,60000000,2022-06-01 00:00:00,20000000.0,NaN,DealHub offers a complete quote-to-revenue sol...,2014,1
3,ZIMPERIUM,Hitesh ICP 2,240.0,computer & network security,Austin,Texas,United States,NaN,Zimperium is the only mobile security platform...,"Akamai DNS, Gmail, Google Apps, Zendesk, Digit...",70878093,Venture (Round not Specified),12000000,2018-11-16 00:00:00,50000000.0,NaN,Zimperium enables companies to realize the ful...,2010,1
4,"Mobilicis, Inc.",NaN,180.0,information technology & services,Austin,Texas,United States,NaN,NaN,NaN,NaN,NaN,NaN,NaN,75000000.0,NaN,Mobilicis is a global leader in Mobile Phone a...,2020,0


In [403]:
# file_path = "/users/amulya/Desktop/Tam_3004.xlsx"
# test.to_excel(file_path, index=False)

In [387]:
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix, roc_curve, roc_auc_score
from sklearn.ensemble import RandomForestClassifier

y = train_final['ICP']
x = train_final.drop('ICP',axis = 1)
RF_model = RandomForestClassifier(criterion = 'log_loss', max_depth= None, min_samples_leaf = 2, min_samples_split = 2, n_estimators = 200)
RF_model.fit(x, y)
y_pred = RF_model.predict(x)
analysis(y, y_pred)

accuracy : 0.8683838097933767
precision:0.973744292237443
recall:0.6586872586872586
f1_score:0.7858129894058038


In [388]:
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix, roc_curve, roc_auc_score

test_backup_final = test_final['ICP'].copy()

y = test_final['ICP']
x = test_final.drop('ICP',axis = 1)
y_pred_test = RF_model.predict(x)
analysis(y, y_pred_test)

accuracy : 0.727735368956743
precision:0.7701149425287356
recall:0.43506493506493504
f1_score:0.5560165975103735


In [400]:
# from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix, roc_curve, roc_auc_score


# test_final.columns = test_final.columns.astype(str)

# x = test_final
# y_pred_test = RF_model.predict(x)
# print(y_pred_test)
# test['predicted_output'] = y_pred_test
# test.head()


/opt/homebrew/anaconda3/envs/amulya_base/lib/python3.9/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(


[0 0 1 ... 0 0 0]


,Company,Lists,Employees,Industry,Company City,Company State,Company Country,Keywords,SEO Description,Technologies,Total Funding,Latest Funding,Latest Funding Amount,Last Raised At,Annual Revenue,Number of Retail Locations,Short Description,Founded Year,predicted_output
0,Southwestern Benefit Designers,NaN,6.0,information technology & services,Austin,Texas,United States,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,PATH Software,"Brain 30K SAAS USA, data companies 24",5.0,information technology & services,Austin,Texas,United States,"solutions for window, door dealerships, busine...",NaN,"Gmail, Google Apps, MailChimp SPF, Google Clou...",NaN,NaN,NaN,NaN,402000.0,NaN,PATH | WD was born out of an observation that ...,2010,0
2,DealHub.io,"Brain 30K SAAS USA, Dhanush/Apollo/Feb 22/2023...",260.0,information technology & services,Austin,Texas,United States,"cpq, proposals, guided selling, sales quoting,...",Streamline your CLM contract management. Win d...,"Cloudflare DNS, Mailchimp Mandrill, Gmail, Goo...",84500000,Series C,60000000,2022-06-01 00:00:00,20000000.0,NaN,DealHub offers a complete quote-to-revenue sol...,2014,1
3,ZIMPERIUM,Hitesh ICP 2,240.0,computer & network security,Austin,Texas,United States,NaN,Zimperium is the only mobile security platform...,"Akamai DNS, Gmail, Google Apps, Zendesk, Digit...",70878093,Venture (Round not Specified),12000000,2018-11-16 00:00:00,50000000.0,NaN,Zimperium enables companies to realize the ful...,2010,1
4,"Mobilicis, Inc.",NaN,180.0,information technology & services,Austin,Texas,United States,NaN,NaN,NaN,NaN,NaN,NaN,NaN,75000000.0,NaN,Mobilicis is a global leader in Mobile Phone a...,2020,0


In [401]:
# file_path = "/users/amulya/Desktop/Tam_9k.xlsx"
# test.to_excel(file_path, index=False)

In [341]:
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score
y = train_final['ICP']
X = train_final.drop('ICP', axis=1)
nb_classifier = GaussianNB()
nb_classifier.fit(X, y)
y_pred_train = nb_classifier.predict(X)
analysis(y, y_pred_train)

accuracy : 0.827625247664874
precision:0.6801470588235294
recall:1.0
f1_score:0.8096280087527352


In [342]:
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score
y = test_final['ICP']
X = test_final.drop('ICP', axis=1)
y_pred_test = nb_classifier.predict(X)

analysis(y, y_pred_test)

accuracy : 0.4758269720101781
precision:0.40225563909774437
recall:0.6948051948051948
f1_score:0.5095238095238095


In [389]:
import xgboost as xgb
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score
y = train_final['ICP']
X = train_final.drop('ICP', axis=1)
xgb_classifier = xgb.XGBClassifier(colsample_bytree = 0.5,degree = 1, gamma = 2.870346165312448 , kernel= 'poly', learning_rate = 0.01 , max_depth = 10 , n_estimators = 222 ,reg_alpha = 0.19149688983851168, reg_lambda = 0.38837431005558154, subsample = 0.8182619559238651)
xgb_classifier.fit(X, y)
y_pred_train = xgb_classifier.predict(X)
analysis(y, y_pred_train)


/opt/homebrew/anaconda3/envs/amulya_base/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [15:47:23] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:742: 
Parameters: { "degree", "kernel" } are not used.

  warnings.warn(smsg, UserWarning)


accuracy : 0.838947070478347
precision:0.8903225806451613
recall:0.6393822393822394
f1_score:0.7442696629213483


In [34]:
import xgboost as xgb
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer
y = train_final['ICP']
X = train_final.drop('ICP', axis=1)

# Step 3: Scale the features
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

param_space = {
    'max_depth': Integer(3, 10),  # Maximum depth of a tree
    'learning_rate': Real(0.01, 1.0, 'log-uniform'),  # Learning rate
    'n_estimators': Integer(100, 1000),  # Number of boosting rounds
    'subsample': Real(0.5, 1.0),  # Subsample ratio of the training instances
    'colsample_bytree': Real(0.5, 1.0),  # Subsample ratio of columns when constructing each tree
    'gamma': Real(0, 5),  # Minimum loss reduction required to make a further partition
    'reg_alpha': Real(0, 1),  # L1 regularization term on weights
    'reg_lambda': Real(0, 1),  # L2 regularization term on weights
    'degree': (1, 6),
    'kernel': ['linear', 'rbf', 'poly']
}
xgb_classifier = xgb.XGBClassifier()

opt = BayesSearchCV(xgb_classifier, param_space, n_iter=50, cv=5, random_state=42)

opt.fit(X_scaled, y)

best_params = opt.best_params_
best_estimator = opt.best_estimator_
# Step 9: Evaluate the best model on the test set
# y_pred_test = best_estimator.predict(X_test_scaled)
# accuracy = accuracy_score(y_test, y_pred_test)
# print("Best Parameters:", best_params)
# print("Test Accuracy:", accuracy)


/opt/homebrew/anaconda3/envs/amulya_base/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [12:41:44] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:742: 
Parameters: { "degree", "kernel" } are not used.

  warnings.warn(smsg, UserWarning)
/opt/homebrew/anaconda3/envs/amulya_base/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [12:41:47] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:742: 
Parameters: { "degree", "kernel" } are not used.

  warnings.warn(smsg, UserWarning)
/opt/homebrew/anaconda3/envs/amulya_base/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [12:41:50] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:742: 
Parameters: { "degree", "kernel" } are not used.

  warnings.warn(smsg, UserWarning)
/opt/homebrew/anaconda3/envs/amulya_base/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [12:41:52] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:742: 
Parameters: { "degree", "kern

In [35]:
print(best_estimator)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.5, degree=1, device=None,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, feature_types=None, gamma=2.870346165312448,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, kernel='poly', learning_rate=0.01,
              max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=10, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=222, n_jobs=None, ...)


In [36]:
print(best_params)

OrderedDict([('colsample_bytree', 0.5), ('degree', 1), ('gamma', 2.870346165312448), ('kernel', 'poly'), ('learning_rate', 0.01), ('max_depth', 10), ('n_estimators', 222), ('reg_alpha', 0.19149688983851168), ('reg_lambda', 0.38837431005558154), ('subsample', 0.8182619559238651)])


In [390]:
import xgboost as xgb
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score
y = test_final['ICP']
X = test_final.drop('ICP', axis=1)
y_pred_test = xgb_classifier.predict(X)
analysis(y, y_pred_test)

accuracy : 0.7557251908396947
precision:0.7636363636363637
recall:0.5454545454545454
f1_score:0.6363636363636364


In [404]:
# import xgboost as xgb
# from sklearn.preprocessing import MinMaxScaler
# from sklearn.metrics import accuracy_score
# # y = test_final['ICP']
# X = test_final
# # scaler = MinMaxScaler()
# # X_scaled = scaler.fit_transform(X)
# y_pred_test = xgb_classifier.predict(X)
# print(y_pred_test)
# test['predicted_output'] = y_pred_test
# test.head()
# analysis(y, y_pred_test)

[0 0 1 ... 0 0 0]


,Company,Lists,Employees,Industry,Company City,Company State,Company Country,Keywords,SEO Description,Technologies,Total Funding,Latest Funding,Latest Funding Amount,Last Raised At,Annual Revenue,Number of Retail Locations,Short Description,Founded Year,predicted_output
0,Southwestern Benefit Designers,NaN,6.0,information technology & services,Austin,Texas,United States,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,PATH Software,"Brain 30K SAAS USA, data companies 24",5.0,information technology & services,Austin,Texas,United States,"solutions for window, door dealerships, busine...",NaN,"Gmail, Google Apps, MailChimp SPF, Google Clou...",NaN,NaN,NaN,NaN,402000.0,NaN,PATH | WD was born out of an observation that ...,2010,0
2,DealHub.io,"Brain 30K SAAS USA, Dhanush/Apollo/Feb 22/2023...",260.0,information technology & services,Austin,Texas,United States,"cpq, proposals, guided selling, sales quoting,...",Streamline your CLM contract management. Win d...,"Cloudflare DNS, Mailchimp Mandrill, Gmail, Goo...",84500000,Series C,60000000,2022-06-01 00:00:00,20000000.0,NaN,DealHub offers a complete quote-to-revenue sol...,2014,1
3,ZIMPERIUM,Hitesh ICP 2,240.0,computer & network security,Austin,Texas,United States,NaN,Zimperium is the only mobile security platform...,"Akamai DNS, Gmail, Google Apps, Zendesk, Digit...",70878093,Venture (Round not Specified),12000000,2018-11-16 00:00:00,50000000.0,NaN,Zimperium enables companies to realize the ful...,2010,1
4,"Mobilicis, Inc.",NaN,180.0,information technology & services,Austin,Texas,United States,NaN,NaN,NaN,NaN,NaN,NaN,NaN,75000000.0,NaN,Mobilicis is a global leader in Mobile Phone a...,2020,0


In [405]:
# file_path = "/users/amulya/Desktop/Tam_xg.xlsx"
# test.to_excel(file_path, index=False)

In [391]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score
y = train_final['ICP']
X = train_final.drop('ICP', axis=1)
adb = AdaBoostClassifier() 
adb.fit(X,y)
y_pred_train = adb.predict(X)
analysis(y, y_pred_train)

/opt/homebrew/anaconda3/envs/amulya_base/lib/python3.9/site-packages/sklearn/ensemble/_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


accuracy : 0.7679026323238042
precision:0.7303588748787585
recall:0.5814671814671815
f1_score:0.647463456577816


In [392]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score
y = test_final['ICP']
X = test_final.drop('ICP', axis=1)
y_pred_test = adb.predict(X)
analysis(y, y_pred_test)

accuracy : 0.7557251908396947
precision:0.7230769230769231
recall:0.6103896103896104
f1_score:0.6619718309859155


In [357]:
# from sklearn.ensemble import AdaBoostClassifier
# from sklearn.preprocessing import MinMaxScaler
# from sklearn.metrics import accuracy_score
# # y = test_final['ICP']
# X = test_final
# # scaler = MinMaxScaler()
# # X_train_scaled = scaler.fit_transform(X)
# y_pred_test = adb.predict(X)
# print(y_pred_test)
# analysis(y, y_pred_test)

[0 0 0 0 1 0 0 0 1 0 0 0 1 0 1 1 0 1 1 0 0 0 1 1 1 1 0 0 0 1 0 0 0 0 0 1 1
 1 1 0 0 0 0 1 0 0]


/opt/homebrew/anaconda3/envs/amulya_base/lib/python3.9/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but AdaBoostClassifier was fitted without feature names
  warnings.warn(


In [347]:
from sklearn.svm import SVC
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score
y = train_final['ICP']
X = train_final.drop('ICP', axis=1)
svm_model = SVC()
svm_model.fit(X, y)
y_pred_train = svm_model.predict(X)
analysis(y, y_pred_train)

accuracy : 0.9247098782904047
precision:0.9830985915492958
recall:0.8084942084942085
f1_score:0.8872881355932203


In [ ]:
from sklearn.svm import SVC
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer
y = train_final['ICP']
X = train_final.drop('ICP', axis=1)

# Step 3: Scale the features
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

param_space = {
    # 'C': Real(1e-6, 100.0, 'log-uniform'),    # Regularization parameter
    # 'kernel': Categorical(['linear', 'rbf']),  # Kernel type
    'gamma': Real(1e-6, 10.0, 'log-uniform'),  # Kernel coefficient for 'rbf' and 'poly' kernels
    # 'degree': Integer(1, 6),  
}
svm_classifier = SVC()

opt = BayesSearchCV(svm_classifier, param_space, n_iter=50, cv=5, random_state=42)

opt.fit(X_scaled, y)

best_params = opt.best_params_
best_estimator = opt.best_estimator_

In [348]:
from sklearn.svm import SVC
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score
y = test_final['ICP']
X = test_final.drop('ICP', axis=1)
y_pred_test = svm_model.predict(X)
analysis(y, y_pred_test)


accuracy : 0.7557251908396947
precision:0.75
recall:0.564935064935065
f1_score:0.6444444444444445


In [359]:
# from sklearn.svm import SVC
# from sklearn.preprocessing import MinMaxScaler
# from sklearn.metrics import accuracy_score
# # y = test_final['ICP']
# X = test_final
# # Scale the features
# # scaler = MinMaxScaler()
# # X_test_scaled = scaler.fit_transform(X)

# # Predict the labels for test data
# y_pred_test = svm_model.predict(X)
# print(y_pred_test)
# analysis(y, y_pred_test)

/opt/homebrew/anaconda3/envs/amulya_base/lib/python3.9/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but SVC was fitted without feature names
  warnings.warn(


[0 0 0 0 1 0 0 0 0 0 0 0 1 1 0 1 0 0 1 0 0 0 0 1 1 0 1 0 0 1 0 0 0 0 0 0 0
 1 0 0 0 0 0 1 1 0]


train accuracy - 0.9947368421052631 - when I'm applying grid search cv(C = 100, fit_intercept = False, max_iter = 400, penalty = 'l1', solver = 'liblinear')
test - test accuracy: 0.6421052631578947